### combine pfak data seg into npy and tiff, getting weight ready

Liya Ding, 2024.06

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)

from skimage.morphology import binary_opening, binary_closing, binary_dilation
from skimage.morphology import disk


from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization


In [2]:
# build the colormap with iterative tab10
tab10 = cm.get_cmap('tab10', 10)
for i in range(10):
    tab10.colors = np.concatenate([tab10.colors,tab10.colors],axis=0)    
tab10.colors = np.concatenate([np.zeros([1,4]),tab10.colors],axis=0)

newmap = cm.get_cmap('tab10', 4000+1)
newmap.colors = tab10.colors[0:4000+1,:]
newmap.colors[0,:]  = np.zeros([1,4])
newmap.colors[0,3] = 1

In [5]:
image_2D_2CH_dir =  '/mnt/d/lding/zyxin/data/240423_fixed_EGFP_zyxin_pFAK/Y27_images'
normed_image_2D_2CH_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/normed_Y27_images'

if not os.path.isdir(normed_image_2D_2CH_dir):   os.makedirs(normed_image_2D_2CH_dir)



In [8]:
# get a global intensity threshold 
filenames = [x for x in os.listdir(image_2D_2CH_dir) if os.path.isfile(os.path.join(image_2D_2CH_dir, x)) and ('.tif' in x)]

for filenameID in range(len(filenames)):
    filename = filenames[filenameID]
    # print(filename)
    img = tifffile.imread(os.path.join(image_2D_2CH_dir,filename))
    pfak_img = img[1,:,:]
    zyxin_img = img[0,:,:]
    
    intensity_scaling_param = [5,20]
    norm_pax_img = intensity_normalization(pax_img, scaling_param=intensity_scaling_param)
    norm_zyxin_img = intensity_normalization(zyxin_img, scaling_param=intensity_scaling_param)
       
    MIP_pax_img = norm_pax_img
    MIP_zyxin_img = norm_zyxin_img

    smooth_MIP_zyxin_img = gaussian_filter(MIP_zyxin_img,sigma=4,mode='nearest',truncate=3)
    ### low threshold to get cell masks
    new_cell_mask = smooth_MIP_zyxin_img>0.02
    ### remove isolated small objects (small regions due to noise)
    new_cell_mask = remove_small_objects(new_cell_mask>0, min_size=10000, connectivity=1, in_place=False)        
    # remove holes
    new_cell_mask = ndimage.binary_fill_holes(new_cell_mask)

    if filenameID ==0:
        all_pax_img = np.zeros([len(filenames),pax_img.shape[0],pax_img.shape[1]])
        all_actin_img = np.zeros([len(filenames),pax_img.shape[0],pax_img.shape[1]])
        all_cell_mask = np.zeros([len(filenames),pax_img.shape[0],pax_img.shape[1]])
    all_pax_img[filenameID,:,:] = pax_img
    all_actin_img[filenameID,:,:] = zyxin_img
    all_cell_mask[filenameID,:,:] = new_cell_mask

    
global_pax_threshold  = all_pax_img[all_cell_mask>0].mean() + 1*all_pax_img[all_cell_mask>0].std()
global_zyxin_threshold  = all_actin_img[all_cell_mask>0].mean() + 1*all_actin_img[all_cell_mask>0].std()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/Y27_pfak/Vessp5-LocThrk1_seg/seg/SS_pax_seg_Y27_zxyin_pFAK_cell_055.tif_MIP.tif'

In [ ]:
# get a global intensity threshold 
filenames = [x for x in os.listdir(image_2D_2CH_dir) if os.path.isfile(os.path.join(image_2D_2CH_dir, x)) and ('.tif' in x)]

for filenameID in range(len(filenames)):
    filename = filenames[filenameID]
    # print(filename)
    img = tifffile.imread(os.path.join(image_2D_2CH_dir,filename))
    zyxin_img = img[0,:,:]
    
    intensity_scaling_param = [5,20]
    norm_pax_img = intensity_normalization(pax_img, scaling_param=intensity_scaling_param)
    norm_zyxin_img = intensity_normalization(zyxin_img, scaling_param=intensity_scaling_param)
       
    MIP_pax_img = norm_pax_img
    MIP_zyxin_img = norm_zyxin_img

    smooth_MIP_zyxin_img = gaussian_filter(MIP_zyxin_img,sigma=4,mode='nearest',truncate=3)
    ### low threshold to get cell masks
    new_cell_mask = smooth_MIP_zyxin_img>0.02
    ### remove isolated small objects (small regions due to noise)
    new_cell_mask = remove_small_objects(new_cell_mask>0, min_size=10000, connectivity=1, in_place=False)        
    # remove holes
    new_cell_mask = ndimage.binary_fill_holes(new_cell_mask)

    if filenameID ==0:
        all_pax_img = np.zeros([len(filenames),pax_img.shape[0],pax_img.shape[1]])
        all_actin_img = np.zeros([len(filenames),pax_img.shape[0],pax_img.shape[1]])
        all_cell_mask = np.zeros([len(filenames),pax_img.shape[0],pax_img.shape[1]])
    all_pax_img[filenameID,:,:] = pax_img
    all_actin_img[filenameID,:,:] = zyxin_img
    all_cell_mask[filenameID,:,:] = new_cell_mask

    
global_pax_threshold  = all_pax_img[all_cell_mask>0].mean() + 1*all_pax_img[all_cell_mask>0].std()
global_zyxin_threshold  = all_actin_img[all_cell_mask>0].mean() + 1*all_actin_img[all_cell_mask>0].std()